In [ ]:
# libraries
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
# set up dataframes & lists for indexing
info_cols = ['link','address','beds','baths','sqft']
history_cols = ['address','Date','Event','Price']

history_df = pd.DataFrame(columns = history_cols)
info_df = pd.DataFrame(columns = info_cols)

props = ['og:zillow_fb:address','zillow_fb:beds','zillow_fb:baths']
labels = ['address','beds','baths','sqft','link']

In [ ]:
# get list of houses
houses = list(pd.read_csv('/Users/lelee1/Desktop/D590TS/homes.csv')['link'])

In [ ]:
for url in houses:
    browser = webdriver.Chrome(executable_path='/Users/lelee1/Desktop/D590TS/chromedriver')
    browser.get(url)
    soup = BeautifulSoup(browser.page_source)
    browser.close()
    try:
        for x in soup.find_all('meta', {'property':"og:zillow_fb:address"},'content'):
            address = x.get('content')


        concat_events = [tr.get('label') for tr in soup.find_all('tr') if tr.get('label') is not None]

        for row in concat_events:
            new = []
            new_row = row.split(', ')
            for x in new_row:
                if ':' in x:
                    new.append(map(str.strip,x.split(':',1)))
            new = dict(new)
            new['address'] = address
            history_df = history_df.append(new, ignore_index = True)
    
        info = []
        for p in props:
            for x in soup.find_all('meta',{'property':p},'content'):
                info.append(x.get('content'))
        info.append(''.join(re.findall('([0-9])',soup.find_all('span',{'data-testid':"bed-bath-beyond"})[0].find_all('span')[6].text)))
        info.append(url)
        info_new = {labels[i]:info[i] for i in range(len(labels))}
        info_df = info_df.append(info_new, ignore_index=True)
    except:
        pass

In [ ]:
# dataframes to csv
history_df.to_csv('historical_initial.csv')
info_df.to_csv('info_initial.csv')